From a couple of different sources we can create a huge dump of 'concrete nouns':

https://www.writing.com/main/view_item/item_id/1757079-Concrete-Nouns-List  
http://ogden.basic-english.org/words.html  
https://www.manythings.org/vocabulary/lists/c/

In [2]:
tmp_list = []
with open("name_dump.txt",'r') as f:
    for line in f:
        for word in word_tokenize(line):
            if len(word) > 1 and re.match('^[\w]+$', word):
                tmp_list.append(word.lower())
                
word_set = set(tmp_list)
                
for word in tmp_list:
    plural = word + 's'
    if plural in word_set:
        word_set.remove(plural)
            
with open("nouns/en/nounlist.txt",'w') as f:
    for word in sorted(word_set):
        f.write(word + '\n')

Count frequency in a dump of wikipedia

In [7]:
import pyspark
import re
from nltk import word_tokenize

sc = pyspark.SparkContext('local[*]')

noun_set = set()
with open("nounlist.txt",'r') as f:
    for line in f:
        noun_set.add(line.strip().lower())
noun_set_bc = sc.broadcast(noun_set)

lines = sc.textFile("enwiki-20190201-pages-articles.xml.bz2")
# lines = sc.parallelize(lines.take(200000)) # in case you want to try a small sample

content_rdd = lines.filter(lambda line: line and line[0].isalpha())
counts = content_rdd.flatMap(lambda line: [word.lower() for word in word_tokenize(line)]) \
            .filter(lambda word: word in noun_set_bc.value) \
            .map(lambda word: (word, 1)) \
            .reduceByKey(lambda a, b: a + b)
count_tuples = counts.sortBy(lambda a: -a[1]).collect()

words_found = []
with open("nounlist_frequency.csv",'w') as f:
    for word,count in count_tuples:
        f.write(word + ',' + str(count) + '\n')
        words_found.append(word)
    
    # make sure all the words are in the file even if never found on wikipedia
    words_found = set(words_found)
    for word in noun_set:
        if word not in words_found:
            f.write(word + ',0\n')
sc.stop()

finally we do some cleaning on the frequency list. Stop words etc should rise to the top and be easy to remove. similiarly, typos should be right at the bottom